In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import sklearn.ensemble
import sklearn.metrics
import matplotlib.pyplot as plt


In [2]:
data=pd.read_csv('features.csv', index_col='match_id')
data.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
missing_data=data.isnull().sum()
missing_data[missing_data>0].sort_values(ascending=False)

first_blood_player2            43987
radiant_flying_courier_time    27479
dire_flying_courier_time       26098
first_blood_player1            19553
first_blood_team               19553
first_blood_time               19553
dire_bottle_time               16143
radiant_bottle_time            15691
radiant_first_ward_time         1836
dire_first_ward_time            1826
radiant_courier_time             692
dire_courier_time                676
dtype: int64

***Пропуск данных***:
***
1. **first_blood_player2**: игры, в которых второй игрок не участвовал в совершении события "первая кровь".
2. **radiant_flying_courier_time, dire_flying_courier_time**: эти значения указывают на то, что никто из игроков соотвествующей(radiant и dire) команды не купил предмет "flying_courier" в первые 5 минут.
3. **first_blood_player1, first_blood_team, first_blood_time**: эти значения одинаковы и указывают на то, что в первые 5 минут вообще не произошло события "первая кровь".
4. **dire_bottle_time, radiant_bottle_time**: эти значения указывают на то, что никто из соотвествующей(dire и radiant) команд не купил предмет "bottle" в первые 5 минут.
5. **radiant_first_ward_time, dire_first_ward_time**: эти значения указывают на то, что никто из игроков соотвествующей(radiant и dire) команды не поставил "наблюдателя" в первые 5 минут.
6. **radiant_courier_time, dire_courier_time**: эти значения указывают на то, что никто из игроков соотвествующей(radiant и dire) команды не купил предмет "courier" в первые 5 минут.
***

***
**Целевой переменной является *radiant_win*, поскольку этот столбец описывает какая команда выиграла.** 
***

In [4]:
#удаляем признаки, связанные с итогами матча и разделяем данные на признаки и целевую переменную
y=data['radiant_win']
X=data.drop(['duration','radiant_win','tower_status_radiant',
             'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
#заменяем пустые значения
X=X.fillna(0)

# Градиентный бустинг " в лоб"

In [5]:
kf=sklearn.model_selection.KFold(n_splits=5,random_state=241,shuffle=True)
results=[]

In [6]:
number=[5,10,20,30,40,50,100,200]
for i in number:
    gb=sklearn.ensemble.GradientBoostingClassifier(n_estimators=i,random_state=241)
    print('n_estimators: {:}'.format(i))
    start_time = datetime.datetime.now()
    cr_val=sklearn.model_selection.cross_val_score(gb,X,y,cv=kf,scoring='roc_auc',n_jobs=-1)    
    print ('Time elapsed: {:}'.format(datetime.datetime.now() - start_time))
    print ('Mean score: {:0.3}'.format(cr_val.mean()))
    results.append(cr_val.mean())

n_estimators: 5
Time elapsed: 0:00:19.889251
Mean score: 0.636
n_estimators: 10
Time elapsed: 0:00:31.079426
Mean score: 0.664
n_estimators: 20
Time elapsed: 0:01:02.935409
Mean score: 0.683
n_estimators: 30
Time elapsed: 0:01:36.276509
Mean score: 0.689
n_estimators: 40
Time elapsed: 0:02:02.267080
Mean score: 0.694
n_estimators: 50
Time elapsed: 0:02:21.119450
Mean score: 0.697
n_estimators: 100
Time elapsed: 0:04:18.416053
Mean score: 0.706
n_estimators: 200
Time elapsed: 0:09:47.164735
Mean score: 0.714


***
**Кросс-валидация для градиентного бустинга с 30 деревьями заняла 1 минуту 36 секунд, метрика качества AUC-ROC составила 0.689**
***

***
**При увеличении числа деревьев улучшается точность: так при 200 деревьях она равна 0.714, когда при 30 - всего 0.689. Но при этом сильно увеличивается время обучения. **
# Меры по ускорению обучения
1. Использовать не всю выборку, а только некоторую её часть.
2. Ограничить глубину деревьев(параметр max_depth).
***